#ライブラリインポート


In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import cv2
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time
import subprocess as sp
from datetime import datetime, timedelta, timezone
import math
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import imageio
vgg19 = models.vgg19(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


#GPU使用時間と種類を表示

In [ ]:
res = sp.Popen(["cat", "/proc/uptime"], stdout=sp.PIPE)
    # 単位はHour
use_time = float(sp.check_output(["awk", "{print $1 /60 /60 }"], stdin=res.stdout).decode().replace("\n",""))
print(use_time)

In [ ]:
!nvidia-smi

#マウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#パラメータ定義

In [ ]:
%cd drive/My\ Drive
dataroot3="DiversityDesign_data/color_pants"
dataroot4="DiversityDesign_data/verygood_smis_pants_mask"
dataroot7="DiversityDesign_data/color_skirt"
dataroot8="DiversityDesign_data/verygood_smis_skirt_mask"
dataroot9="DiversityDesign_data/color_tops"
dataroot10="DiversityDesign_data/verygood_smis_tops_mask"
dataroot11="DiversityDesign_data/color_persons"
num_thread=0
batch_size=16
num_epoch=15
img_size=(128,96)
lr=0.0002
b1=0.5
b2=0.999
lamda1=6.85 
lamda2=2.9
lamda3=1.1
ngpu=1

/content/drive/My Drive


#データのロード


In [ ]:
!unzip DiversityDesign_data.zip

In [ ]:
!unzip color_data.zip

In [ ]:
pants_dataset=dset.ImageFolder(root=dataroot3,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
pants_mask_dataset=dset.ImageFolder(root=dataroot4,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
skirt_dataset=dset.ImageFolder(root=dataroot7,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
skirt_mask_dataset=dset.ImageFolder(root=dataroot8,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
tops_dataset=dset.ImageFolder(root=dataroot9,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
tops_mask_dataset=dset.ImageFolder(root=dataroot10,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
person_dataset=dset.ImageFolder(root=dataroot11,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))

pants_dataloader=torch.utils.data.DataLoader(pants_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)    
pants_mask_dataloader=torch.utils.data.DataLoader(pants_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
skirt_dataloader=torch.utils.data.DataLoader(skirt_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
skirt_mask_dataloader=torch.utils.data.DataLoader(skirt_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
tops_dataloader=torch.utils.data.DataLoader(tops_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
tops_mask_dataloader=torch.utils.data.DataLoader(tops_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
person_dataloader=torch.utils.data.DataLoader(person_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)

device=torch.device("cuda:0")

#関数定義

##切り抜き

In [ ]:
def overlay(p,f,tops,pants,skirt):
  Mask=(tops+pants+skirt)
  Mask=MaskTrans(Mask)
  reverse=1-Mask
  cutout1=p*reverse
  cutout2=Mask*f
  out=cutout1+cutout2
  return out

##mask閾値

In [ ]:
def  MaskTrans(mask):
  for i in range(mask.size(0)):
    for j in range(mask.size(1)):
      for k in range(mask.size(2)):
        for l in range(mask.size(3)):
          if mask[i][j][k][l]>0.6:
            mask[i][j][k][l]=1
          else:
            mask[i][j][k][l]=0
  return mask

##重みの初期化

In [ ]:
def init_weights(model):
  if isinstance(model.modules,nn.Conv2d):
      model.modules().weight.data.nomal_(0,0.002)
      model.modules().bias.data.zero_()
  if isinstance(model.modules,nn.ConvTranspose2d):
      model.modules().weight.data.nomal_(0,0.002)
      model.modules().bias.data.zero_()

##KLDloss


In [ ]:
def KLDloss(mu, logvar):
  lamda=0.05
  return lamda*(-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()))

#クラス定義


##Encoder-Decoder



In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self,ngpu):
    super(EncoderDecoder,self).__init__()
    self.ngpu=ngpu
    #Encoder
    df=3
    self.down1_1=nn.Conv2d(3*df,32*df,3,2,1,bias=False,groups=3)
    self.innorm1_1=nn.InstanceNorm2d(32*df,eps=1e-5)
    self.Leaky=nn.LeakyReLU(0.2,inplace=True)
    self.down1_2=nn.Conv2d(32*df,64*df,3,2,1,bias=False,groups=3)
    self.innorm1_2=nn.InstanceNorm2d(64*df,eps=1e-5)
    self.down1_3=nn.Conv2d(64*df,128*df,3,2,1,bias=False,groups=3)
    self.innorm1_3=nn.InstanceNorm2d(128*df,eps=1e-5)
    self.down1_4=nn.Conv2d(128*df,256*df,3,2,1,bias=False,groups=3)
    self.innorm1_4=nn.InstanceNorm2d(256*df,eps=1e-5)
    self.gamma=nn.Conv2d(256*df,2*df,(8,6),1,0,bias=False,groups=3)
    self.beta=nn.Conv2d(256*df,2*df,(8,6),1,0,bias=False,groups=3)
    
    #Decoder
    self.down1_0=nn.Conv2d(2*df,256*df,3,1,1,groups=3)
    self.relu=nn.ReLU()
    self.leaky=nn.LeakyReLU(0.2,inplace=True)
    self.batch2_1=nn.BatchNorm2d(256*df,eps=1e-5,affine=False)
    self.pool2_1=nn.MaxPool2d(16)
    self.share2_1=nn.Conv2d(df*1,16*df,3,1,1,groups=3)
    self.gamma2_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.beta2_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.gconv2_1=nn.Conv2d(256*df,256*df,3,1,1,groups=3)
    self.up2_1=nn.Upsample(scale_factor=2,mode="nearest")
    self.batch3_1=nn.BatchNorm2d(256*df,eps=1e-5,affine=False)
    self.pool3_1=nn.MaxPool2d(8)
    self.share3_1=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.gamma3_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.beta3_1=nn.Conv2d(16*df,256*df,3,1,1,groups=3)
    self.gconv3_1=nn.Conv2d(256*df,128*df,3,1,1,groups=3)
    self.up3_1=nn.Upsample(scale_factor=2,mode="nearest")
    self.batch4_1=nn.BatchNorm2d(128*df,eps=1e-5,affine=False)
    self.pool4_1=nn.MaxPool2d(4)
    self.share4_1=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.gamma4_1=nn.Conv2d(16*df,128*df,3,1,1,groups=3)
    self.beta4_1=nn.Conv2d(16*df,128*df,3,1,1,groups=3)
    self.gconv4_1=nn.Conv2d(128*df,64*df,3,1,1,groups=3)
    self.up4_1=nn.Upsample(scale_factor=4,mode="nearest")
    self.smooth4_1=nn.Conv2d(64*df,64*df,3,1,1,groups=3)
    self.batch5_1=nn.BatchNorm2d(64*df,eps=1e-5,affine=False)
    self.pool5_1=nn.MaxPool2d(1)
    self.share5_1=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.gamma5_1=nn.Conv2d(16*df,64*df,3,1,1,groups=3)
    self.beta5_1=nn.Conv2d(16*df,64*df,3,1,1,groups=3)
    self.gconv5_1=nn.Conv2d(64*df,32*df,3,1,1,groups=3)
    self.pbatch=nn.BatchNorm2d(32*df,eps=1e-5,affine=False)
    self.ppool=nn.MaxPool2d(1)
    self.pshare=nn.Conv2d(1*df,16*df,3,1,1,groups=3)
    self.pgamma=nn.Conv2d(16*df,32*df,3,1,1,groups=3)
    self.pbeta=nn.Conv2d(16*df,32*df,3,1,1,groups=3)
    self.pgconv=nn.Conv2d(32*df,20*df,3,1,1,groups=3)
    self.lastconv=nn.Conv2d(20*df,3,1,1,0)
    self.sigmoid=nn.Sigmoid()

  def Encoder(self,input): 
    down1_1=self.down1_1(input)
    innorm1_1=self.innorm1_1(down1_1)

    leaky1_2=self.Leaky(innorm1_1)
    down1_2=self.down1_2(leaky1_2)
    innorm1_2=self.innorm1_2(down1_2)
  
    leaky1_3=self.Leaky(innorm1_2)
    down1_3=self.down1_3(leaky1_3)
    innorm1_3=self.innorm1_3(down1_3)
    
    leaky1_4=self.Leaky(innorm1_3)
    down1_4=self.down1_4(leaky1_4)
    innorm1_4=self.innorm1_4(down1_4)
    
    leaky1_5=self.Leaky(innorm1_4)
    gamma=self.gamma(leaky1_5)
    beta=self.beta(leaky1_5)

    return gamma,beta

  def Decoder(self,z,seg,test=False): 
    down1_0=self.down1_0(z)

    batch2_1=self.batch2_1(down1_0)
    share2_1=self.relu(self.share2_1(self.pool2_1(seg)))
    gamma2_1=self.gamma2_1(share2_1)
    beta2_1=self.beta2_1(share2_1)
    temp2_1=(batch2_1*gamma2_1)+beta2_1
    out2_1=self.up2_1(self.gconv2_1(self.leaky(temp2_1)))
  
    batch3_1=self.batch3_1(out2_1)
    share3_1=self.relu(self.share3_1(self.pool3_1(seg)))
    gamma3_1=self.gamma3_1(share3_1)
    beta3_1=self.beta3_1(share3_1)
    temp3_1=(batch3_1*gamma3_1)+beta3_1
    out3_1=self.up3_1(self.gconv3_1(self.leaky(temp3_1)))
  
    batch4_1=self.batch4_1(out3_1)
    share4_1=self.relu(self.share4_1(self.pool4_1(seg)))
    gamma4_1=self.gamma4_1(share4_1)
    beta4_1=self.beta4_1(share4_1)
    temp4_1=(batch4_1*gamma4_1)+beta4_1
    out4_1=self.up4_1(self.gconv4_1(self.leaky(temp4_1)))
    out4_1=self.smooth4_1(out4_1)
  
    batch5_1=self.batch5_1(out4_1)
    share5_1=self.relu(self.share5_1(self.pool5_1(seg)))
    gamma5_1=self.gamma5_1(share5_1)
    beta5_1=self.beta5_1(share5_1)
    temp5_1=(batch5_1*gamma5_1)+beta5_1
    out5_1=self.gconv5_1(self.leaky(temp5_1))
    
    batch=self.pbatch(out5_1)
    share=self.relu(self.pshare(self.ppool(seg)))
    gamma=self.pgamma(share)
    beta=self.pbeta(share)
    temp=(batch*gamma)+beta
    out=self.pgconv(self.leaky(temp))
    last=self.leaky(out)   
    out5=self.sigmoid(self.lastconv(last))

    return out5,last


  def reparameterize(self,mu, logvar):
      std = torch.exp(0.5 * logvar) 
      eps = torch.randn_like(std)/4
      return eps.mul(std) + mu


  def forward(self,input,seg,Z,test=False,pre=False,encodeonly=False):
    if not test and not pre and not encodeonly:
      mu,logvar=self.Encoder(input)
      z=self.reparameterize(mu,logvar)
      fake,last=self.Decoder(z,seg)
      return fake,mu,logvar,z,last
    
    elif pre==True:
      fake,_=self.Decoder(Z,seg,test=True)
      return fake

    elif encodeonly==True:
      mu,logvar=self.Encoder(input)
      z=self.reparameterize(mu,logvar)
      return z,mu,logvar

    else:
      z=torch.randn((1,6,1,1)).to(device)
      z=z.repeat(seg.size(0),1,1,1)
      fake,_=self.Decoder(z,seg,test=True)
      return fake,z

##Discriminator

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,ngpu):    
    super(Discriminator,self).__init__()
    self.ngpu=ngpu
    df=3
    dc=df+3
    
    self.pool=nn.AvgPool2d(2)
    self.down1_1=nn.Conv2d(dc,8*dc,4,2,2)
    self.leaky=nn.LeakyReLU(0.2)
    self.down1_2=nn.Conv2d(8*dc,16*dc,4,2,2)
    self.innorm1_2=nn.InstanceNorm2d(16*dc)
    self.down1_3=nn.Conv2d(16*dc,32*dc,4,2,2)
    self.innorm1_3=nn.InstanceNorm2d(32*dc)
    self.down1_4=nn.Conv2d(32*dc,64*dc,4,1,2)
    self.innorm1_4=nn.InstanceNorm2d(64*dc)
    self.down1_5=nn.Conv2d(64*dc,1,4,1,2)
    self.sigmoid=nn.Sigmoid()

  def Discriminator(self,image,seg): 
    featmap=[]
    data=torch.cat([image,seg],dim=1)
    
    pool1_1=self.pool(data)
    down1_1=self.down1_1(pool1_1)
    leaky1_1=self.leaky(down1_1)
    featmap.append(leaky1_1)
   
    down1_2=self.down1_2(leaky1_1)
    innorm1_2=self.innorm1_2(down1_2)
    leaky1_2=self.leaky(innorm1_2)
    featmap.append(leaky1_2)
  
    down1_3=self.down1_3(leaky1_2)
    innorm1_3=self.innorm1_3(down1_3)
    leaky1_3=self.leaky(innorm1_3)
    featmap.append(leaky1_3)
    
    down1_4=self.down1_4(leaky1_3)
    innorm1_4=self.innorm1_4(down1_4)
    leaky1_4=self.leaky(innorm1_4)
    featmap.append(leaky1_4)

    down1_5=self.down1_5(leaky1_4)

    return self.sigmoid(down1_5),featmap

##知覚損失(/有)


In [ ]:
class Vgg19Loss(nn.Module):
    def __init__(self):
        super(Vgg19Loss, self).__init__()
        features1=list(vgg19.features)[:3]
        features2=list(vgg19.features)[:8]
        features3=list(vgg19.features)[:13]
        features4=list(vgg19.features)[:22]
        features5=list(vgg19.features)[:31]
        self.features1=nn.ModuleList(features1).eval()
        self.features2=nn.ModuleList(features2).eval()
        self.features3=nn.ModuleList(features3).eval()
        self.features4=nn.ModuleList(features4).eval()
        self.features5=nn.ModuleList(features5).eval()

    def forward(self,x,y):
        lamda=5
        t1=x
        t2=y
        loss1=nn.MSELoss()
        loss2=nn.MSELoss()
        loss3=nn.MSELoss()
        loss4=nn.MSELoss()
        loss5=nn.MSELoss()

        for f in self.features1:
            x=f(x)
            y=f(y)
        f1loss=torch.sqrt(loss1(x,y)/(64*128*96))
      
        x=t1
        y=t2
        for f in self.features2:
            x=f(x)
            y=f(y)
        f2loss=torch.sqrt(loss2(x,y)/(128*64*48))

        x=t1
        y=t2
        for f in self.features3:
            x=f(x)
            y=f(y)
        f3loss=torch.sqrt(loss3(x,y)/(256*32*24))

        x=t1
        y=t2
        for f in self.features4:
            x=f(x)
            y=f(y)
        f4loss=torch.sqrt(loss4(x,y)/(512*16*12))

        x=t1
        y=t2
        for f in self.features5:
            x=f(x)
            y=f(y)
        f5loss=torch.sqrt(loss5(x,y)/(512*8*6))
  
        return (f1loss+f2loss+f3loss+f4loss+f5loss)*lamda

#Encoder-Decoder,Discriminatorの初期化

In [ ]:
netG=EncoderDecoder(ngpu).to(device)
netG.apply(init_weights)
netD=Discriminator(ngpu).to(device)
netD.apply(init_weights)

#Adam最適化＆損失関数

In [ ]:
L1=nn.L1Loss()
Lper=Vgg19Loss().to(device)
Ladv=nn.BCELoss()
tmapL1=nn.L1Loss()
smapL1=nn.L1Loss()
pmapL1=nn.L1Loss()

real_label=1
fake_label=0

optimizernetG=optim.Adam(netG.parameters(),lr=lr,betas=(b1,b2))
optimizernetD=optim.Adam(netD.parameters(),lr=lr,betas=(b1,b2))

#学習

In [ ]:
Generator_losses=[]
Discriminator_losses = []
Sum_losses = []
kld_losses=[]
vgg_losses=[]
feat_losses=[]
L1_losses=[]
judge1=[]
judge2=[]
judge3=[]

In [ ]:
i=0
netG.train()
t1=time.time()
for epoch in range(num_epoch):
   iter_tops = iter(tops_dataloader)
   iter_pants=iter(pants_dataloader)
   iter_skirt = iter(skirt_dataloader)
   iter_person=iter(person_dataloader)

   iter_tops_mask = iter(tops_mask_dataloader)
   iter_pants_mask=iter(pants_mask_dataloader)
   iter_skirt_mask= iter(skirt_mask_dataloader)

   for tops_data in tops_dataloader:

        tops_real_batch=next(iter_tops)
        pants_real_batch=next(iter_pants)
        skirt_real_batch=next(iter_skirt)
        person_real_batch=next(iter_person)
        tops_mask_real_batch=next(iter_tops_mask)
        pants_mask_real_batch=next(iter_pants_mask)
        skirt_mask_real_batch=next(iter_skirt_mask)
       
        tops=tops_real_batch[0].to(device)              
        pants=pants_real_batch[0].to(device)      
        skirt=skirt_real_batch[0].to(device)  
        person=person_real_batch[0].to(device)
        tops_mask=tops_mask_real_batch[0].to(device)              
        pants_mask=pants_mask_real_batch[0].to(device)      
        skirt_mask=skirt_mask_real_batch[0].to(device) 

        tmask=tops_mask.repeat(1,20,1,1) 
        pmask=pants_mask.repeat(1,20,1,1)
        smask=skirt_mask.repeat(1,20,1,1)
        b_size = tops.size(0)
        
        input=torch.cat([tops,pants,skirt],dim=1)
        seg=torch.cat([tops_mask,pants_mask,skirt_mask],dim=1)
        
        for j in range(1):
          
          fake,mu,logvar,_,map=netG(input,seg,_)
    
          #Discriminator
          netD.zero_grad()
          item=tops+pants+skirt
          
          pre_t,featmap_t=netD.Discriminator(item,seg)
          judge1.append(pre_t.mean())
          label = torch.full(pre_t.size(), fill_value=real_label,dtype=torch.float32,device=device)
          D_loss_t = Ladv(pre_t,label)
          D_loss_t.backward()

          pre_f,featmap_f=netD.Discriminator(fake.detach(),seg)
          judge2.append(pre_f.mean())
          label.fill_(fake_label)
          D_loss_f= Ladv(pre_f,label)
          D_loss_f.backward()
          D_loss=D_loss_t+D_loss_f
          optimizernetD.step()
          
          #Generator
          if i%1==0:
            netG.zero_grad()
            label.fill_(real_label)
            pre_Gf,featmap_f=netD.Discriminator(fake,seg)

            G_loss_f = Ladv(pre_Gf,label)
            pre_t,featmap_t=netD.Discriminator(item,seg)
            kld_loss=KLDloss(mu,logvar)
            L1_loss=L1(fake,item)
            vgg_loss=Lper(item,fake)     
            tmap_loss=tmapL1(map[:,:20,:,:],tmask)*lamda1 
            pmap_loss=pmapL1(map[:,20:40,:,:],pmask)*lamda2  
            smap_loss=smapL1(map[:,40:60,:,:],smask)*lamda3  

            kld_losses.append(kld_loss.detach())
            vgg_losses.append(vgg_loss.detach())
            L1_losses.append(L1_loss.detach())
        
            G_loss=G_loss_f+kld_loss+L1_loss+(vgg_loss*3)+tmap_loss+pmap_loss+smap_loss
            G_loss.backward()
            optimizernetG.step()

          judge3.append(pre_Gf.mean())
          Sum_losses.append(G_loss.detach())
          Generator_losses.append(G_loss_f.detach())
          Discriminator_losses.append(D_loss.detach())

#テスト


##再構築テスト



In [ ]:
%cd drive/My\ Drive
dataroot3="DiversityDesign_data/abs_pants"
dataroot4="DiversityDesign_data/test_smis_pants_mask"
dataroot7="DiversityDesign_data/abs_skirt"
dataroot8="DiversityDesign_data/test_smis_skirt_mask"
dataroot9="DiversityDesign_data/abs_tops"
dataroot10="DiversityDesign_data/test_smis_tops_mask"
dataroot11="DiversityDesign_data/abs_person"

num_thread=0
batch_size=10
num_epoch=15
img_size=(128,96)
lr=0.0002
b1=0.5
b2=0.999
ngpu=1

In [ ]:
test_pants_dataset=dset.ImageFolder(root=dataroot3,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
test_pants_mask_dataset=dset.ImageFolder(root=dataroot4,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_skirt_dataset=dset.ImageFolder(root=dataroot7,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
test_skirt_mask_dataset=dset.ImageFolder(root=dataroot8,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_tops_dataset=dset.ImageFolder(root=dataroot9,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
test_tops_mask_dataset=dset.ImageFolder(root=dataroot10,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_person_dataset=dset.ImageFolder(root=dataroot11,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))

test_pants_dataloader=torch.utils.data.DataLoader(test_pants_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)    
test_pants_mask_dataloader=torch.utils.data.DataLoader(test_pants_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_skirt_dataloader=torch.utils.data.DataLoader(test_skirt_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_skirt_mask_dataloader=torch.utils.data.DataLoader(test_skirt_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_tops_dataloader=torch.utils.data.DataLoader(test_tops_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_tops_mask_dataloader=torch.utils.data.DataLoader(test_tops_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_person_dataloader=torch.utils.data.DataLoader(test_person_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)

device=torch.device("cuda:0")

In [ ]:
images=[]
i=0
bre=0
end=0#終了バッチの設定 0~4
netG.eval()
for epoch in range(1):

   iter_tops = iter(test_tops_dataloader)
   iter_pants=iter(test_pants_dataloader)
   iter_skirt = iter(test_skirt_dataloader)
   iter_person=iter(test_person_dataloader)

   iter_tops_mask = iter(test_tops_mask_dataloader)
   iter_pants_mask=iter(test_pants_mask_dataloader)
   iter_skirt_mask= iter(test_skirt_mask_dataloader)
  

   t1=time.time()
   for tops_data in test_tops_dataloader:
        #0.バッチデータの取得     
        tops_real_batch=next(iter_tops)
        pants_real_batch=next(iter_pants)
        skirt_real_batch=next(iter_skirt)
        person_real_batch=next(iter_person)

        tops_mask_real_batch=next(iter_tops_mask)
        pants_mask_real_batch=next(iter_pants_mask)
        skirt_mask_real_batch=next(iter_skirt_mask)
  
       
        tops=tops_real_batch[0].to(device)              #[][][][]
        pants=pants_real_batch[0].to(device)      #[][][][]
        skirt=skirt_real_batch[0].to(device)  
        person=person_real_batch[0].to(device)

        tops_mask=tops_mask_real_batch[0].to(device)              #[][][][]
        pants_mask=pants_mask_real_batch[0].to(device)      #[][][][]
        skirt_mask=skirt_mask_real_batch[0].to(device)  
       

        b_size = tops.size(0)#バッチサイズを計算
        
        
        #データの連結
        input=torch.cat([tops,pants,skirt],dim=1)
        seg=torch.cat([tops_mask,pants_mask,skirt_mask],dim=1)
        
        #fakeを生成
        if bre==end:
            z,mu,logvar=netG(input,_,_,encodeonly=True)
            fake=netG(_,seg,z,pre=True)
            fake_image=overlay(person,fake,tops_mask,pants_mask,skirt_mask)
            true_image=overlay(person,tops+pants+skirt,tops_mask,pants_mask,skirt_mask)
            images.append(fake_image.detach())
            break      
        bre+=1

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(np.transpose(vutils.make_grid(fake_image.detach().to(device)[:10], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(np.transpose(vutils.make_grid(true_image.detach().to(device)[:10], padding=2, normalize=True).cpu(),(1,2,0)))

##多様性テスト

In [ ]:
%cd drive/My\ Drive

dataroot3="DiversityDesign_data/yellow_pants"
dataroot4="DiversityDesign_data/test_smis_pants_mask"
dataroot7="DiversityDesign_data/yellow_skirt"
dataroot8="DiversityDesign_data/test_smis_skirt_mask"
dataroot9="DiversityDesign_data/yellow_tops"
dataroot10="DiversityDesign_data/test_smis_tops_mask"
dataroot11="DiversityDesign_data/abs_person"
num_thread=0
batch_size=1
num_epoch=15
img_size=(128,96)
lr=0.0002
b1=0.5
b2=0.999
ngpu=1

In [ ]:
test_pants_dataset=dset.ImageFolder(root=dataroot3,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
test_pants_mask_dataset=dset.ImageFolder(root=dataroot4,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_skirt_dataset=dset.ImageFolder(root=dataroot7,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
test_skirt_mask_dataset=dset.ImageFolder(root=dataroot8,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_tops_dataset=dset.ImageFolder(root=dataroot9,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))
test_tops_mask_dataset=dset.ImageFolder(root=dataroot10,transform=transforms.Compose([transforms.Resize(img_size),transforms.Grayscale(),transforms.ToTensor(),]))
test_person_dataset=dset.ImageFolder(root=dataroot11,transform=transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),]))

test_pants_dataloader=torch.utils.data.DataLoader(test_pants_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)    
test_pants_mask_dataloader=torch.utils.data.DataLoader(test_pants_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_skirt_dataloader=torch.utils.data.DataLoader(test_skirt_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_skirt_mask_dataloader=torch.utils.data.DataLoader(test_skirt_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_tops_dataloader=torch.utils.data.DataLoader(test_tops_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_tops_mask_dataloader=torch.utils.data.DataLoader(test_tops_mask_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)
test_person_dataloader=torch.utils.data.DataLoader(test_person_dataset,batch_size=batch_size,shuffle=False,num_workers=num_thread)

device=torch.device("cuda:0")

In [ ]:
netG.eval()
for epoch in range(1):
   iter_tops = iter(test_tops_dataloader)
   iter_pants=iter(test_pants_dataloader)
   iter_skirt = iter(test_skirt_dataloader)

   for tops_data in test_tops_dataloader:
        #0.バッチデータの取得     
        tops_real_batch=next(iter_tops)
        pants_real_batch=next(iter_pants)
        skirt_real_batch=next(iter_skirt)
       
        tops=tops_real_batch[0].to(device)              #[][][][]
        pants=pants_real_batch[0].to(device)      #[][][][]
        skirt=skirt_real_batch[0].to(device)  

        b_size = tops.size(0)#バッチサイズを計算
          
        #データの連結
        input=torch.cat([tops,pants,skirt],dim=1)
       
        #fakeを生成
        z,mu,logvar=netG(input,_,_,encodeonly=True)
        print(z.size())
        print(z.mean())

In [ ]:
z=z.repeat(batch_size,1,1,1)

In [ ]:
n=3
z[:,0,:,:]=-n
z[:,1,:,:]=n
z[:,2,:,:]=0
z[:,3,:,:]=0
z[:,4,:,:]=0
z[:,5,:,:]=0

In [ ]:
images=[]
i=0
count=0
t=7
for i in range(n*2*2):
  z[:,1,:,:]=z[:,1,:,:]-0.5
  z[:,0,:,:]=-n
  for epoch in range(n*2*2):
    z[:,0,:,:]=z[:,0,:,:]+0.5
    iter_person=iter(test_person_dataloader)

    iter_tops_mask = iter(test_tops_mask_dataloader)
    iter_pants_mask=iter(test_pants_mask_dataloader)
    iter_skirt_mask= iter(test_skirt_mask_dataloader)
    
    t1=time.time()
    count=0
    for tops_data in test_tops_mask_dataloader:
          #0.バッチデータの取得     
          person_real_batch=next(iter_person)
          tops_mask_real_batch=next(iter_tops_mask)
          pants_mask_real_batch=next(iter_pants_mask)
          skirt_mask_real_batch=next(iter_skirt_mask)

          person=person_real_batch[0].to(device)
          tops_mask=tops_mask_real_batch[0].to(device)              #[][][][]
          pants_mask=pants_mask_real_batch[0].to(device)      #[][][][]
          skirt_mask=skirt_mask_real_batch[0].to(device)  
          
          b_size = tops.size(0)#バッチサイズを計算
          #print(b_size)
          
          
          #データの連結
          seg=torch.cat([tops_mask,pants_mask,skirt_mask],dim=1)
          
         
          #fakeを生成
          if count==t:
            fake=netG(_,seg,z,pre=True) #衣服のみ
          #fake_image=overlay(person,fake,tops_mask,pants_mask,skirt_mask) #人物重ね合わせ
            images.append(fake.detach())
          count+=1
      

In [ ]:
total=0
fig=plt.figure(figsize=(90,90))
for i in range(n*2*2):
  for j in range(n*2*2):
    plt.subplot(n*2*2,n*2*2,total+1)
    plt.imshow(np.transpose(vutils.make_grid(images[total], padding=2, normalize=True).cpu(),(1,2,0)))
    total+=1
plt.savefig("image.png")
plt.show()

#損失を表示

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training",fontsize=23)
plt.plot(Generator_losses,label="G")
plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations",fontsize=23)
plt.ylabel("Loss",fontsize=23)
plt.tick_params(labelsize=17)
#plt.ylim(0,10)
#plt.xticks(np.arange(0,27001,10000))
#plt.yticks(np.arange(0,11,5))
plt.legend()
plt.savefig("graphg.png")
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("Sum Loss During Training")
plt.plot(Sum_losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig("graph02")
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("kld Loss During Training")
plt.plot(kld_losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()
print(kld_losses[-1])

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("L1 Loss During Training")
plt.plot(L1_losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.savefig("graph02.png")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("L1 Loss During Training")
plt.plot(tmapL1Losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
#plt.savefig("graph02")
plt.legend()
plt.show()

In [ ]:
#@title
fig=plt.figure(figsize=(10,5))
plt.title("L1 Loss During Training")
plt.plot(pmapL1Losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
#plt.savefig("graph02")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("L1 Loss During Training")
plt.plot(smapL1Losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
#plt.savefig("graph02")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("vgg Loss During Training")
plt.plot(vgg_losses,label="Unet")
#plt.plot( Discriminator_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.savefig("graph03")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("Unet Loss During Training")
plt.plot(judge1,label="Dt")
plt.plot(judge2,label="Df")
plt.plot(judge3,label="Gf")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

#退避

In [ ]:
%cd ..

In [ ]:
%cd drive/My\ Drive

In [ ]:
def avoid_G():    
    PATHUnet='drive/My Drive/DiversityDesign_data/model/Encoder_Decoder2.pth'
    torch.save({
            'epoch': epoch,
            #'image':img_list,
            'Sum_losses':Sum_losses,
            'Generator':Generator_losses,
            'Vgg':vgg_losses,
            'kld':kld_losses,
            'L1':L1_losses,
            'judge1':judge1,
            'judge2':judge2,
            'judge3':judge3,
            'model_state_dict': netG.state_dict(),
            'optimizer_state_dict': optimizernetG.state_dict(),
            'criterion1':Ladv.state_dict(),
            'criterion2':L1.state_dict(),
            'criterion3':Lper.state_dict()

            }, PATHUnet)
def avoid_D():
    PATHD='drive/My Drive/DiversityDesign_data/models/Discriminator2.pth'
    torch.save({
            'Disciriminator_losses':Discriminator_losses,
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizernetD.state_dict()
            }, PATHD)

In [ ]:
avoid_G()
avoid_D()

#ロード

In [ ]:
PATHG='drive/My Drive/DiversityDesign_data/model/Encoder_Decoder.pth'
#Unet = EncoderDecoder(ngpu).to(device)
optimizernetG= optim.Adam(netG.parameters(),lr=lr,betas=(b1,b2))
Ladv=nn.BCELoss()
L1=nn.L1Loss()
Lper=Vgg19Loss().to(device)

checkpoint = torch.load(PATHG)
netG.load_state_dict(checkpoint['model_state_dict'])
optimizernetG.load_state_dict(checkpoint['optimizer_state_dict'])
Ladv.load_state_dict(checkpoint['criterion1'])
L1.load_state_dict(checkpoint['criterion2'])
Lper.load_state_dict(checkpoint['criterion3'])
epoch = checkpoint['epoch']
#img_list=checkpoint['image']
Sum_losses=checkpoint['Sum_losses']
Generator_losses=checkpoint['Generator']
kld_losses=checkpoint['kld']
vgg_losses=checkpoint['Vgg']
L1_losses=checkpoint["L1"]
judge1=checkpoint['judge1']
judge2=checkpoint['judge2']
judge3=checkpoint['judge3']

In [ ]:
PATHD='drive/My Drive/DiversityDesign_data/model/Discriminator.pth'

optimizernetD=optim.Adam(netD.parameters(),lr=lr,betas=(b1,b2))


checkpoint = torch.load(PATHD)
netD.load_state_dict(checkpoint['model_state_dict'])
optimizernetD.load_state_dict(checkpoint['optimizer_state_dict'])
Discriminator_losses=checkpoint['Disciriminator_losses']